In [63]:
import numpy as np
import scipy.misc
import keras.backend as K
import tensorflow as tf
import sys

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils, plot_model
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.initializers import glorot_normal
from matplotlib.pyplot import imshow
from blocks import *
from res_model import *
from data_generator import DataGenerator
from keras.backend.tensorflow_backend import set_session
from dataset_utils import *

In [64]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

In [65]:
class_to_id = get_class_to_id()
X_train, X_val = get_train_val()
Y_train, Y_val = get_labels(class_to_id, X_train, X_val)

params = {'dim': (150, 150),
          'batch_size': 32,
          'n_classes': 24,
          'n_channels': 3,
          'shuffle': True}

train_generator = DataGenerator(X_train, Y_train, **params)
val_generator = DataGenerator(X_val, Y_val, **params)

In [66]:

model = ResNet32(input_shape=(150, 150, 3), classes=params['n_classes'])
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=0.000001)
checkpoint_saver = ModelCheckpoint('resnet32.h5', verbose=True)
tensorboard_logger = TensorBoard('logs',
                                 histogram_freq=0,
                                 write_graph=False,
                                 write_grads=False,
                                 batch_size=32,
                                 write_images=False)

callback_list = [lr_reducer, checkpoint_saver, tensorboard_logger]

# Normalize image vectors

# Convert training and test labels to one hot matrices
print("number of training examples = " + str(len(X_train)))
print("number of test examples = " + str(len(X_val)))
print("X_train shape: " + str(len(X_train)))
print("Y_train shape: " + str(len(Y_train)))
print("X_test shape: " + str(len(X_val)))
print("Y_test shape: " + str(len(Y_val)))

model.summary()

number of training examples = 52992
number of test examples = 12782
X_train shape: 52992
Y_train shape: 52992
X_test shape: 12782
Y_test shape: 12782
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256  

In [67]:
hist = model.fit_generator(
    generator=train_generator,
    validation_data=val_generator,
    callbacks=callback_list,
    use_multiprocessing=True,
    workers=3, epochs=30, verbose=True)

Epoch 1/30
1656/1656 [==============================] - 404s 244ms/step - loss: 2.1705 - acc: 0.5885 - top_k_categorical_accuracy: 0.8747 - val_loss: 6.7044 - val_acc: 0.3171 - val_top_k_categorical_accuracy: 0.9032

Epoch 00001: saving model to resnet32.h5
Epoch 2/30
1656/1656 [==============================] - 403s 244ms/step - loss: 0.3449 - acc: 0.9175 - top_k_categorical_accuracy: 0.9942 - val_loss: 2.5707 - val_acc: 0.5343 - val_top_k_categorical_accuracy: 0.9241

Epoch 00002: saving model to resnet32.h5
Epoch 3/30
1656/1656 [==============================] - 402s 243ms/step - loss: 0.0935 - acc: 0.9700 - top_k_categorical_accuracy: 0.9993 - val_loss: 2.2459 - val_acc: 0.5588 - val_top_k_categorical_accuracy: 0.9091

Epoch 00003: saving model to resnet32.h5
Epoch 4/30
1656/1656 [==============================] - 410s 248ms/step - loss: 0.0676 - acc: 0.9782 - top_k_categorical_accuracy: 0.9995 - val_loss: 1.5888 - val_acc: 0.6931 - val_top_k_categorical_accuracy: 0.9478

Epoch 000

In [69]:
model = load_model('resnet32.h5')
preds = model.evaluate_generator(
    val_generator, workers=6, use_multiprocessing=True)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))
with open('history.txt','w') as f:
    f.write(str(hist))

Loss = 1.9106995618124878
Test Accuracy = 0.787045739348371


In [70]:
import cv2

In [71]:
id_to_sign = {v:k for k,v in class_to_id.items()}

In [72]:
model = load_model('snapshots/resnet32_30.h5')

In [75]:
img = cv2.imread('../experiment 2/dataset_massey/hand2_a_dif_seg_5_cropped.png',cv2.IMREAD_COLOR)
# img = cv2.imread('raw_dataset/B/t/color_19_0152.png')
img = cv2.resize(img,(150,150),interpolation=cv2.INTER_LINEAR)
img = img / 255
lst = np.empty((1,150,150,3))
lst[0] = img
res = model.predict(lst,verbose=True)


1/1 [==============================] - 0s 22ms/step


In [76]:
print([(round(el,5),id_to_sign[i]) for i,el in enumerate(res[0])])
print(res)

[(0.0, 'a'), (0.0, 'b'), (0.0, 'c'), (0.0, 'd'), (0.0, 'e'), (0.0, 'f'), (0.0, 'g'), (0.0, 'h'), (0.0, 'i'), (0.0, 'k'), (0.0, 'l'), (0.0, 'm'), (0.0, 'n'), (0.0, 'o'), (0.0, 'p'), (0.0, 'q'), (0.0, 'r'), (0.0, 's'), (0.0, 't'), (0.0, 'u'), (0.0, 'v'), (0.0, 'w'), (0.0, 'x'), (1.0, 'y')]
[[2.9003326e-18 1.4768559e-15 1.7550211e-15 6.8696489e-29 9.5266255e-20
  1.5057262e-22 5.8592009e-22 9.1206656e-26 5.4822879e-23 2.1617318e-20
  1.0792125e-25 3.0291052e-18 5.3053023e-19 3.4429054e-13 5.2149759e-16
  3.3359814e-08 3.0872712e-25 3.3862167e-11 9.4358441e-09 2.0199924e-09
  4.4537521e-24 5.1079209e-19 1.5505271e-11 1.0000000e+00]]
